In [2]:
!pip install "apache-airflow==2.0.1" --constraint "https://raw.githubusercontent.com/apache/airflow/constraints-2.0.1/constraints-3.6.txt"

     |████████████████████████████████| 4.5MB 5.8MB/s 
     |████████████████████████████████| 61kB 4.8MB/s 
     |████████████████████████████████| 61kB 4.8MB/s 
     |████████████████████████████████| 81kB 6.0MB/s 
     |████████████████████████████████| 3.2MB 35.0MB/s 
     |████████████████████████████████| 993kB 22.0MB/s 
     |████████████████████████████████| 122kB 24.8MB/s 
     |████████████████████████████████| 296kB 40.0MB/s 
     |████████████████████████████████| 163kB 50.4MB/s 
     |████████████████████████████████| 174kB 46.9MB/s 
     |████████████████████████████████| 143kB 45.5MB/s 
     |████████████████████████████████| 1.8MB 41.9MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
     |████████████████████████████████| 163kB 45.0MB/s 
     |████████████████████████████████| 102kB 9.4MB/s 
     |████████████████████████████████| 133kB 38.0MB/s 
     |████████████████████████████████| 512kB 41.8MB/s 
     |████████████████████████████████| 645kB 31.7MB/s 
  

Criando a DAG

In [6]:
from datetime import datetime, timedelta
# The DAG object; we'll need this to instantiate a DAG
from airflow import DAG

In [7]:
# Operators; we need this to operate!
from airflow.operators.bash import BashOperator
from airflow.utils.dates import days_ago
# These args will get passed on to each operator
# You can override them on a per-task basis during operator initialization
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    # Start on 27th of June, 2020
    'start_date': datetime(2020, 6, 27),
    'email': ['airflow@example.com'],
    'email_on_failure': False,
    'email_on_retry': False,
    # In case of errors, do one retry
    'retries': 1,
    # Do the retry with 30 seconds delay after the error
    'retry_delay': timedelta(seconds=30),
    # Run once every 15 minutes
    'schedule_interval': '*/15 * * * *'
}
dag = DAG(
    'tutorial',
    default_args=default_args,
    description='A simple tutorial DAG',
    schedule_interval=timedelta(days=1),
    start_date=days_ago(2),
    tags=['example'],
)

# t1, t2 and t3 are examples of tasks created by instantiating operators
t1 = BashOperator(
    task_id='print_date',
    bash_command='date',
    dag=dag,
)

t2 = BashOperator(
    task_id='sleep',
    depends_on_past=False,
    bash_command='sleep 5',
    retries=3,
    dag=dag,
)
dag.doc_md = __doc__

t1.doc_md = """\
#### Task Documentation
You can document your task using the attributes `doc_md` (markdown),
`doc` (plain text), `doc_rst`, `doc_json`, `doc_yaml` which gets
rendered in the UI's Task Instance Details page.
![img](http://montcs.bloomu.edu/~bobmon/Semesters/2012-01/491/import%20soul.png)
"""
templated_command = """
{% for i in range(5) %}
    echo "{{ ds }}"
    echo "{{ macros.ds_add(ds, 7)}}"
    echo "{{ params.my_param }}"
{% endfor %}
"""

t3 = BashOperator(
    task_id='templated',
    depends_on_past=False,
    bash_command=templated_command,
    params={'my_param': 'Parameter I passed in'},
    dag=dag,
)

t1 >> [t2, t3]

[<Task(BashOperator): sleep>, <Task(BashOperator): templated>]

In [8]:
# initialize the database tables
!airflow db init

# print the list of active DAGs
!airflow dags list

# prints the list of tasks in the "tutorial" DAG
!airflow tasks list tutorial

# prints the hierarchy of tasks in the "tutorial" DAG
!airflow tasks list tutorial --tree

DB: sqlite:////root/airflow/airflow.db
[2021-03-17 18:54:32,462] {db.py:674} INFO - Creating tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144 UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
INFO  [alembic.runtime.migration] Running upgrade  -> e3a246e0dc1, current schema
INFO  [alembic.runtime.migration] Running upgrade e3a246e0dc1 -> 1507a7289a2f, create is_encrypted
/usr/local/lib/python3.7/dist-packages/alembic/ddl/sqlite.py:44 UserWarning: Skipping unsupported ALTER for creation of implicit constraintPlease refer to the batch mode feature which allows for SQLite migrations using a copy-and-move strategy.
INFO  [alembic.runtime.migrati

In [9]:
!airflow tasks test tutorial print_date 2015-06-01


[2021-03-17 18:54:55,238] {dagbag.py:448} INFO - Filling up the DagBag from /root/airflow/dags
[2021-03-17 18:54:55,249] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): create_entry_group>, delete_entry_group already registered
[2021-03-17 18:54:55,249] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): delete_entry_group>, create_entry_group already registered
[2021-03-17 18:54:55,249] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): create_entry_gcs>, delete_entry already registered
[2021-03-17 18:54:55,249] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): delete_entry>, create_entry_gcs already registered
[2021-03-17 18:54:55,249] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): create_tag>, delete_tag already registered
[2021-03-17 18:54:55,249] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): delete_tag>, create_tag already registered
[2021-03-17 18:54:55,259] {baseoperator.py:1153} WARN

In [10]:
!airflow tasks test tutorial sleep 2015-06-01

[2021-03-17 18:55:08,268] {dagbag.py:448} INFO - Filling up the DagBag from /root/airflow/dags
[2021-03-17 18:55:08,279] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): create_entry_group>, delete_entry_group already registered
[2021-03-17 18:55:08,279] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): delete_entry_group>, create_entry_group already registered
[2021-03-17 18:55:08,279] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): create_entry_gcs>, delete_entry already registered
[2021-03-17 18:55:08,280] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): delete_entry>, create_entry_gcs already registered
[2021-03-17 18:55:08,280] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): create_tag>, delete_tag already registered
[2021-03-17 18:55:08,280] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): delete_tag>, create_tag already registered
[2021-03-17 18:55:08,288] {baseoperator.py:1153} WARN

In [11]:
!airflow tasks test tutorial templated 2015-06-01


[2021-03-17 18:55:19,043] {dagbag.py:448} INFO - Filling up the DagBag from /root/airflow/dags
[2021-03-17 18:55:19,054] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): create_entry_group>, delete_entry_group already registered
[2021-03-17 18:55:19,054] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): delete_entry_group>, create_entry_group already registered
[2021-03-17 18:55:19,054] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): create_entry_gcs>, delete_entry already registered
[2021-03-17 18:55:19,054] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): delete_entry>, create_entry_gcs already registered
[2021-03-17 18:55:19,054] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): create_tag>, delete_tag already registered
[2021-03-17 18:55:19,055] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): delete_tag>, create_tag already registered
[2021-03-17 18:55:19,062] {baseoperator.py:1153} WARN

In [14]:
# Operators; we need this to operate!
from airflow.operators.bash import BashOperator
from airflow.utils.dates import days_ago
# These args will get passed on to each operator
# You can override them on a per-task basis during operator initialization
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    # Start on 27th of June, 2020
    'start_date': datetime(2020, 6, 27),
    'email': ['airflow@example.com'],
    'email_on_failure': False,
    'email_on_retry': False,
    # In case of errors, do one retry
    'retries': 1,
    # Do the retry with 30 seconds delay after the error
    'retry_delay': timedelta(seconds=30),
    # Run once every 15 minutes
    'schedule_interval': '*/15 * * * *'
}
with DAG(    dag_id='simple_bash_dag',
    default_args=default_args,
    schedule_interval=None,
    tags=['my_dags'],
) as dag:    #Here we define our first task
    t1 = BashOperator(bash_command="touch ~/my_bash_file.txt", task_id="create_file")    #Here we define our second task
    t2 = BashOperator(bash_command="mv ~/my_bash_file.txt ~/my_bash_file_changed.txt", task_id="change_file_name")    # Configure T2 to be dependent on T1’s execution
    t1 >> t2

In [15]:
!airflow scheduler


[2021-03-17 18:57:44,171] {dagbag.py:448} INFO - Filling up the DagBag from /root/airflow/dags
[2021-03-17 18:57:44,182] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): create_entry_group>, delete_entry_group already registered
[2021-03-17 18:57:44,182] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): delete_entry_group>, create_entry_group already registered
[2021-03-17 18:57:44,182] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): create_entry_gcs>, delete_entry already registered
[2021-03-17 18:57:44,182] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): delete_entry>, create_entry_gcs already registered
[2021-03-17 18:57:44,183] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): create_tag>, delete_tag already registered
[2021-03-17 18:57:44,183] {baseoperator.py:1153} WARNING - Dependency <Task(BashOperator): delete_tag>, create_tag already registered
[2021-03-17 18:57:44,191] {baseoperator.py:1153} WARN